In [ ]:
'''
import sys
sys.path.append('/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/')
from code_blocks.kernels.periodic_inputs_maternkernel import PeriodicInputsMaternKernel
from code_blocks.likelihoods.gaussian_likelihood import GaussianLikelihood
from gpytorch.kernels import MaternKernel
import torch

original_kernel = MaternKernel(nu=2.5)
test_kernel = PeriodicInputsMaternKernel(nu=2.5, period=0.33)
test_likelihood = GaussianLikelihood()
'''

In [ ]:
'''
test_tensor = torch.tensor([0.5, 1, 2, 3])
test_kernel(test_tensor).to_dense()
x1 = 2 * test_tensor * torch.pi / 0.33
x1 = torch.cat((torch.sin(x1).reshape(-1, 1), torch.cos(x1).reshape(-1, 1)), axis=1)
original_kernel(x1).to_dense()
'''

In [ ]:
'''
original_kernel.lengthscale = 15.
for name, params in original_kernel.named_parameters():
    print(name)
    print(params)
'''

In [ ]:
'''
import yaml

root_config = '/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/configs/' 
# NOTE: Specify name here for different experiments: 
# rnd (fix) + unfix (fix) ; first referring to initialization, second referring to inducing points in input space 
curr_config_name = 'spatiotemp/Scale_Matern52_plus_Scale_PeriodicInputsMatern52/lvmogp_catlatent_rnd_unfix' 
curr_config = f'{root_config}/{curr_config_name}.yaml'
with open(curr_config, 'r') as file: 
    config = yaml.safe_load(file) 

import sys
sys.path.append('/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/')
from code_blocks.likelihoods.gaussian_likelihood import GaussianLikelihood
from gplvm_init import GPLVM, train_gplvm
from run_experiments.prepare_dataset import prepare_spatio_temp_data
import torch


_, data_Y_squeezed, *arg = prepare_spatio_temp_data(config=config)

data_Y = data_Y_squeezed.reshape(config['n_outputs'], config['n_input'])[:, :config['n_input_train']]

class GaussianLikelihoodWithMissingObs(GaussianLikelihood):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @staticmethod
    def _get_masked_obs(x):
        missing_idx = x.isnan()
        x_masked = x.masked_fill(missing_idx, -999.)
        return missing_idx, x_masked

    def expected_log_prob(self, target, input, *params, **kwargs):
        missing_idx, target = self._get_masked_obs(target)
        res = super().expected_log_prob(target, input, *params, **kwargs)
        return res * ~missing_idx

    def log_marginal(self, observations, function_dist, *params, **kwargs):
        missing_idx, observations = self._get_masked_obs(observations)
        res = super().log_marginal(observations, function_dist, *params, **kwargs)
        return res * ~missing_idx
    
my_GPLVM = GPLVM(n = config['n_outputs'], 
                 data_dim= config['n_input_train'], 
                 latent_dim = 2, 
                 n_inducing = 20)

my_likelihood = GaussianLikelihoodWithMissingObs()

_, _, losses = train_gplvm(my_GPLVM, my_likelihood, data_Y)

import matplotlib.pyplot as plt
plt.plot(losses)
'''

# Test cached function

In [ ]:
from gpytorch.utils.memoize import cached

In [ ]:
class class_test():

    def __init__(self, a=1.):
        self.a = a
    
    # @property
    @cached(name='value_b')
    def value_b(self):
        return self.a ** 2

class_test = class_test(a=10.)
print(class_test.value_b())
print(class_test._memoize_cache)
class_test.a = 11.
print(class_test.a)
print(class_test.value_b())
print(class_test._memoize_cache)

class class_test_2():

    def __init__(self, a=1.):
        self.a = a
    
    # @property
    @cached(name='value_b')
    def value_b(self, c):
        return self.a ** c
    
class_test_2 = class_test_2(a=10.)
print(class_test_2.value_b(c=2.))
print(class_test_2._memoize_cache)

class_test_2.a = 11.

print(class_test_2.value_b(c=2.))
print(class_test_2._memoize_cache)

print(class_test_2.value_b(c=3.))
print(class_test_2._memoize_cache)

import pickle

pickle.dumps({'c': 2.})